## Change pathogens into the set of taxa you want to predict

In [ ]:
pathogens = set() # Taxa need to be in the form of URIs, i.e. <http://purl.obolibrary.org/obo/NCBITaxon_11676>
# For example, we want to predict for Zika virus
pathogens.add('<http://purl.obolibrary.org/obo/NCBITaxon_64320>')

## Loading data the same way as in train.ipynb

In [ ]:
from __future__ import print_function
bacteria = set()
virus = set()
species_dict = {}
with open('data/ncbitaxon/categories.dmp', 'r') as f:
    for line in f:
        items = line.strip().split('\t')
        species_dict['<http://purl.obolibrary.org/obo/NCBITaxon_'+items[2]+'>'] = '<http://purl.obolibrary.org/obo/NCBITaxon_' + items[1]+'>'
        if items[0] == 'B':
            bacteria.add('<http://purl.obolibrary.org/obo/NCBITaxon_'+items[2]+'>')
        elif items[0] == 'V':
            virus.add('<http://purl.obolibrary.org/obo/NCBITaxon_'+items[2]+'>')
print(len(bacteria), len(virus), len(species_dict))

In [ ]:
import pandas as pd
import numpy as np

import pickle 


embedding_file = 'opa2vec/backup/embeddings_patho_ncbitaxon_short.out'
data = pd.read_csv(embedding_file, header = None, sep = ' ', skiprows=0)
embds_data = data.values
patho_dict = dict(zip(embds_data[:,0],embds_data[:,1:]))
print('finished reading embeddings 1')

embedding_file = 'opa2vec2/backup/embeddings_HPiMPiGO_short.out'
data = pd.read_csv(embedding_file, header = None, sep = ' ', skiprows=0)
embds_data = data.values
host_dict = dict(zip(embds_data[:,0],embds_data[:,1:]))
print('finished reading embeddings 2')

protein_set = set()
with open('host_pheno_asso/human_pheno_asso_HPiMPiGO.txt','r') as f:
    for line in f:
        items = line.strip().split()
        protein_set.add(items[0])
print(len(protein_set))

hosts = set()
pathos = set()
positives_set = set()
count_dict = {}
with open('data/hpidb2.score.txt', 'r') as f:
    for line in f:
        items = line.strip().split('\t')
        patho = '<http://purl.obolibrary.org/obo/NCBITaxon_' + items[1] + '>'
        if ':' in items[2]:
            if float(items[2].split('miscore:')[1]) >= 0.5:
                hosts.add(items[0])
                pathos.add(patho)
                positives_set.add((items[0], patho))
print(len(hosts), len(pathos), len(positives_set))

positives = set()
for pair in positives_set:
    if (pair[0] in host_dict and  pair[0] in protein_set) and (pair[1] in patho_dict and pair[1] in virus):
        positives.add(pair)
print(len(positives))

pathogens_counts = {}
protein_counts = {}
for positive in positives:
    if positive[1] not in pathogens_counts:
        pathogens_counts[positive[1]] = 0
    pathogens_counts[positive[1]]+=1
    if positive[0] not in protein_counts:
        protein_counts[positive[0]] = 0
    protein_counts[positive[0]]+=1
print(len(pathogens_counts), len(protein_counts))

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation
from keras.utils import multi_gpu_model, Sequence, np_utils
import math
from keras.optimizers import SGD, Adam, RMSprop, Nadam
from keras.callbacks import EarlyStopping, TensorBoard
import scipy.stats as ss
import matplotlib.pyplot as plt
from keras.backend.tensorflow_backend import set_session
from keras import backend as K
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

class Generator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        self.nbatch = int(np.ceil(len(self.x) / float(self.batch_size)))
        self.length = len(self.x)

    def __len__(self):
        return self.nbatch

    def __getitem__(self, idx):
        #print('idx: ',idx)
        start = idx * self.batch_size
        batch_len = min(self.batch_size, (self.length)-start)
        X_batch_list = np.empty((batch_len, 600), dtype=np.float32)
        y_batch_list = np.empty(batch_len, dtype=np.float32)
       
        for ids in range(start, min((idx + 1) * self.batch_size, self.length)):
            array1 = host_dict[self.x[ids][0]]
            array2 = patho_dict[self.x[ids][1]]
            embds = np.concatenate([array1, array2])
            X_batch_list[ids-start] = embds
            y_batch_list[ids-start] = self.y[ids]
           
        return X_batch_list, y_batch_list


# In[5]:


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.3
sess = tf.Session(config=config)
K.set_session(sess)

## Training and predicting, saving the results in predictions.tsv

In [ ]:
rank_counts = {}
write_counts = {}
counter = 0
output = open('predictions.tsv', 'w')
output.close()
for taxon in pathogens:
    K.clear_session()
    counter+=1
    print('taxon ', counter)
    val_pathos = set()
    val_pathos.add(taxon)
    train_pathos = set(list(pathogens_counts.keys())) - val_pathos
    print('train pathos: ', len(train_pathos))
    print(val_pathos)
    
    train_positives = []
    val_positives = []
    train_positives_set = set()
    val_positives_set = set()
    for items in positives:
        if items[1] in train_pathos:
            train_positives_set.add((items[0], items[1]))
            train_positives.append((items[0], items[1], 1))
        if items[1] in val_pathos:
            val_positives_set.add((items[0], items[1]))
            val_positives.append((items[0], items[1], 1))
    print(len(train_positives), len(val_positives))

    train_negatives = []
    train_all_tuples = set()

    for host in protein_set:
        for patho in train_pathos:
            if host in host_dict:
                train_all_tuples.add((host, patho))
    print(len(train_all_tuples))

    for item in train_all_tuples:
        if item not in train_positives_set:
            train_negatives.append((item[0], item[1], 0))
    print(len(train_negatives))

    val_negatives = []
    val_all_tuples = set()

    for host in protein_set:
        for patho in val_pathos:
            if host in host_dict:
                val_all_tuples.add((host, patho))
    print(len(val_all_tuples))

    for item in val_all_tuples:
        if item not in val_positives_set:
            val_negatives.append((item[0], item[1], 0))
    print(len(val_negatives))

    train_positives = np.repeat(np.array(list(train_positives)), len(train_negatives)//len(train_positives), axis = 0)
    train_negatives = np.array(train_negatives)
    triple_train = np.concatenate((train_positives, train_negatives), axis=0)
    np.random.shuffle(triple_train)
    triple_val = np.concatenate((val_positives, val_negatives), axis=0)

    batch_size = 2**12
    num_classes = 1

    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(600,)))
    model.add(BatchNormalization())
    model.add(Dropout(0.515))
    model.add(Dense(1, activation='sigmoid'))
    #parallel_model = multi_gpu_model(model, 2)
    parallel_model = model
    parallel_model.compile(loss='binary_crossentropy',
                  optimizer=Adam(),
                  metrics=['accuracy'])

    generator = Generator(triple_train[:len(triple_train),0:2], triple_train[:len(triple_train),2], batch_size)
    val_generator = Generator(triple_val[:,0:2], triple_val[:,2], batch_size)

    num_positive = len(val_positives)
    history = parallel_model.fit_generator(generator=generator,
                        epochs=30,
                        steps_per_epoch = int(math.ceil(len(triple_train)/ batch_size)*0.04),
                        verbose=2,
                        validation_data=generator,
                        validation_steps= 1, max_queue_size = 50,
                        use_multiprocessing=True,
                        workers = 5)
    sim_list = parallel_model.predict_generator(generator=Generator(triple_val[:,0:2], triple_val[:,2], 1000), 
                                                verbose=2, steps=int(math.ceil(math.ceil(len(triple_val))/1000)), 
                                                max_queue_size = 20, use_multiprocessing=True, workers = 3)
    
    with open('predictions.tsv', 'a+') as f:
        for i in range(triple_val.shape[0]):
            f.write('%s\t%s\t%f\n' % (triple_val[i][1],  triple_val[i][0], sim_list[i])) 